In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
from typing import List, Tuple

def trans(x1: float, y1: float,                 # start position
          x2: float, y2: float,                 # target position
          x1v: float, y1v: float,               # start velocity
          x2v: float, y2v: float,               # target velocity
          kp: float = 10.0,                    # PID gains (same for x & y)
          ki: float = 0.0,
          kd: float = 40.0,
          dt: float = 0.01,                    # integration step (s)
          max_steps: int = 100_000) -> List[Tuple[float, float]]:
    """
    Returns a list of (x, y) points forming a smooth, velocity-continuous arc
    from (x1,y1,x1v,y1v) to (x2,y2,x2v,y2v).
    """
    # State variables
    x,  y  = x1,  y1
    vx, vy = x1v, y1v
    ix, iy = 0.0, 0.0                # integral terms

    points: List[Tuple[float, float]] = [(x, y)]

    # Helper to check convergence
    def done() -> bool:
        pos_ok = abs(x - x2) < 0.01 and abs(y - y2) < 0.01
        vel_ok = abs(vx - x2v) < 0.01 and abs(vy - y2v) < 0.01
        return pos_ok and vel_ok

    for _ in range(max_steps):
        if done():
            break

        # --- PID on X --------------------------------------------------------
        ex  = (x2 - x)                         # position error
        evx = (x2v - vx)                       # velocity error (derivative term)
        ix += ex * dt                          # integral
        fx  = kp*ex + ki*ix + kd*evx           # control force  (mass = 1)

        # --- PID on Y --------------------------------------------------------
        ey  = (y2 - y)
        evy = (y2v - vy)
        iy += ey * dt
        fy  = kp*ey + ki*iy + kd*evy

        # --- Physics integration (explicit Euler) ---------------------------
        ax, ay = fx, fy                        # mass = 1 → a = F
        vx += ax * dt
        vy += ay * dt
        x  += vx * dt
        y  += vy * dt

        points.append((x, y))

    return np.array(points)


info = trans(10, 0, 0, 1, 1, 0, 1, 0, kp=5, kd=10)
px.scatter(x=info[:,0], y=info[:,1])


In [ ]:
def bezier_curve(p0, p1, p2, p3, steps):
    pts = []
    for i in range(steps + 1):
        t = i / steps
        u = 1 - t
        x = (u**3)*p0[0] + 3*(u**2)*t*p1[0] + 3*u*(t**2)*p2[0] + (t**3)*p3[0]
        y = (u**3)*p0[1] + 3*(u**2)*t*p1[1] + 3*u*(t**2)*p2[1] + (t**3)*p3[1]
        pts.append((x, y))
    return np.array(pts)

def hermite_curve(p0, p1, t0, t1, steps):
    pts = []
    for i in range(steps + 1):
        s = i / steps
        h00 = 2*s**3 - 3*s**2 + 1
        h10 = s**3 - 2*s**2 + s
        h01 = -2*s**3 + 3*s**2
        h11 = s**3 - s**2
        x = h00*p0[0] + h10*t0[0] + h01*p1[0] + h11*t1[0]
        y = h00*p0[1] + h10*t0[1] + h01*p1[1] + h11*t1[1]
        pts.append((x, y))
    return np.array(pts)

# ---- Bézier spec ---------------------------------------------------
P0 = (10, 0)
P1 = (13, 0)
P2 = (-3, 1)
P3 = (0, 1)
steps = 1000

bezier = bezier_curve(P0, P1, P2, P3, steps)

# ---- Bézier → Hermite tangents -------------------------------------
tension = 5.0            # 0 = flat ends, 1 = Bézier-exact, >1 = overshoot
T0 = 3 * tension * (np.array(P1) - np.array(P0))
T1 = 3 * tension * (np.array(P3) - np.array(P2))

hermite = hermite_curve(P0, P3, T0, T1, steps)

# ---- Optional: inspect / compare -----------------------------------
df = pd.DataFrame({
    "bx": bezier[:,0], "by": bezier[:,1],
    "hx": hermite[:,0], "hy": hermite[:,1],
})

px.scatter(x=df.hx, y=df.hy).show()

In [ ]:
from typing import List, Tuple

def bezier_curve(
    p0: Tuple[float, float],
    p1: Tuple[float, float],
    p2: Tuple[float, float],
    p3: Tuple[float, float],
    steps: int = 256
) -> List[Tuple[float, float]]:
    """
    Generate a cubic Bézier curve from p0 to p3 with control points p1 and p2.

    p0: Start point (x0, y0)
    p1: Control point 1
    p2: Control point 2
    p3: End point (x3, y3)
    steps: Number of interpolation steps
    Returns a list of (x, y) points
    """
    curve = []
    for i in range(steps + 1):
        t = i / steps
        u = 1 - t
        # Cubic Bézier blending
        x = (u**3)*p0[0] + 3*(u**2)*t*p1[0] + 3*u*(t**2)*p2[0] + (t**3)*p3[0]
        y = (u**3)*p0[1] + 3*(u**2)*t*p1[1] + 3*u*(t**2)*p2[1] + (t**3)*p3[1]
        curve.append((x, y))
    return np.array(curve)

def hermite_curve(p0, p1, t0, t1, steps=100):
    """
    Generate points along a cubic Hermite curve.

    Parameters:
    - p0: (x, y) start point
    - p1: (x, y) end point
    - t0: (x, y) tangent at start
    - t1: (x, y) tangent at end
    - steps: number of interpolated points

    Returns:
    - List of (x, y) points along the curve
    """
    points = []
    for i in range(steps + 1):
        t = i / steps
        h00 = 2*t**3 - 3*t**2 + 1
        h10 = t**3 - 2*t**2 + t
        h01 = -2*t**3 + 3*t**2
        h11 = t**3 - t**2

        x = h00*p0[0] + h10*t0[0] + h01*p1[0] + h11*t1[0]
        y = h00*p0[1] + h10*t0[1] + h01*p1[1] + h11*t1[1]

        points.append((x, y))
    return points


info = bezier_curve(
    p0=(10, 0),          # start
    p1=(13, 0),       # control point 1
    p2=(-3, 1),       # control point 2
    p3=(0, 1) ,    # end
    steps=1000
)

t0 = (0, 1) 
t1 = (0, 1) 

curve = hermite_curve((10, 0), (0, 1), t0, t1, steps=999)
diff = np.diff(info, axis=0)
df = pd.DataFrame(np.concatenate([info[1:], diff, curve], axis=1), columns=['x', 'y', 'dx', 'dy', 'hx', 'hy'])
px.scatter(x=df.hx, y=df.hy).show()

In [ ]:
df = df.assign(
    angle=np.arctan2(df.dy, df.dx),
    speed=np.sqrt(df.dx**2 + df.dy**2)
)
df = df.assign(acceleration=np.sqrt(np.diff(df.speed, prepend=df.speed.values[0])**2))

# px.scatter(x=df.angle).show()
px.scatter(x=df.speed).show()
px.scatter(x=df.acceleration).show()

In [ ]:
from typing import List, Tuple
from math import hypot
vec = Tuple[float, float]

def next_point_pid(P1: vec, P2: vec, V1: vec, max_accel: float,
                   Kp: float = 1.0, Ki: float = 0.0, Kd: float = 0.5,
                   integral: vec = (0.0, 0.0), dt: float = 1.0) -> Tuple[vec, vec, vec]:
    # Error = desired_position - current_position
    error = (P2[0] - P1[0], P2[1] - P1[1])

    # Derivative = desired_velocity - current_velocity
    derivative = (-V1[0], -V1[1])  # Assume desired velocity is 0 at target

    # Integrate error
    integral = (integral[0] + error[0] * dt, integral[1] + error[1] * dt)

    # PID output
    ax = Kp * error[0] + Ki * integral[0] + Kd * derivative[0]
    ay = Kp * error[1] + Ki * integral[1] + Kd * derivative[1]

    # Limit total acceleration
    mag = hypot(ax, ay)
    if mag > max_accel:
        ax *= max_accel / mag
        ay *= max_accel / mag

    # New velocity and position
    V2 = (V1[0] + ax * dt, V1[1] + ay * dt)
    P3 = (P1[0] + V2[0] * dt, P1[1] + V2[1] * dt)

    return P3, V2, integral

def meet_paths(start: vec, end: vec,
               v_start: vec = (0., 0.),
               v_end: vec = (0., 0.),
               max_accel: float = 0.05,
               epsilon: float = 0.01,
               max_steps: int = 500) -> List[vec]:
    """
    Generates a single path from `start` to `end` by moving
    two points toward each other with capped acceleration.
    """
    P1, P2 = start, end
    V1, V2 = v_start, v_end

    forward:  List[vec] = [P1]
    backward: List[vec] = [P2]

    for _ in range(max_steps):

        # advance both ends
        P1_next, V1 = next_point(P1, P2, V1, max_accel)
        P2_next, V2 = next_point(P2, P1, V2, max_accel)

        # if they would cross each other, clamp to midpoint and stop
        if hypot(P1_next[0] - P2_next[0], P1_next[1] - P2_next[1]) < epsilon:
            mid = ((P1_next[0] + P2_next[0]) / 2,
                   (P1_next[1] + P2_next[1]) / 2)
            forward.append(mid)
            backward.append(mid)
            break

        # otherwise record and continue
        forward.append(P1_next)
        backward.append(P2_next)
        P1, P2 = P1_next, P2_next

    # stitch: forward  (start→mid) + reversed(backward[1:]) (mid→end)
    backward.reverse()
    backward = backward[1:]                     # drop duplicate mid
    full_path = forward + backward
    return full_path


# demo ---------------------------------------------------------------
# full = meet_paths((10, 0), (0, 1),
#                   v_start=(.15, 0),
#                   v_end=(-.15, 0),
#                   max_accel=0.05, 
#                   epsilon=0., max_steps=1000)

# xs, ys = zip(*full)
# px.line(x=xs, y=ys)
# Start state
p = (10.0, 0.0)      # position
v = (0.15, 0.0)      # velocity
integral = (0.0, 0.0)

# Trajectory buffer
ps = []

# PID gains and accel cap
Kp, Kd, Ki = 0.3, 4, 0.0
Amax = 0.02

for _ in range(100):
    p, v, integral = next_point_pid(
        P1=p,
        P2=(0.0, 1.0),      # target
        V1=v,
        max_accel=Amax,
        Kp=Kp, Kd=Kd, Ki=Ki,
        integral=integral,  # pass the running integral
        dt=1.0              # timestep
    )
    ps.append(p)

px.scatter(x=[pt[0] for pt in ps], y=[pt[1] for pt in ps])


In [252]:
from math import hypot
from typing import List, Tuple

vec = Tuple[float, float]

# --- PID step helper (from previous answer) --------------------------
def limit_vec(x: float, y: float, lim: float) -> vec:
    d = hypot(x, y)
    if d > lim and d != 0.0:
        s = lim / d
        return x * s, y * s
    return x, y

def next_point_pid(P1: vec, P2: vec,
                   V1: vec, integral: vec,
                   max_accel: float,
                   Kp: float, Ki: float, Kd: float,
                   dt: float = 1.0) -> Tuple[vec, vec, vec]:
    err  = (P2[0] - P1[0], P2[1] - P1[1])
    deriv = (-V1[0], -V1[1])               # desired end vel = 0
    integral = (integral[0] + err[0]*dt, integral[1] + err[1]*dt)

    ax = Kp*err[0] + Ki*integral[0] + Kd*deriv[0]
    ay = Kp*err[1] + Ki*integral[1] + Kd*deriv[1]
    ax, ay = limit_vec(ax, ay, max_accel)

    V2 = (V1[0] + ax*dt, V1[1] + ay*dt)
    P3 = (P1[0] + V2[0]*dt, P1[1] + V2[1]*dt)
    return P3, V2, integral
# ---------------------------------------------------------------------


def meet_paths_pid(start: vec, end: vec,
                   v_start: vec = (0., 0.), v_end: vec = (0., 0.),
                   max_accel: float = 0.05,
                   Kp: float = 0.4, Kd: float = 0.15, Ki: float = 0.0,
                   epsilon: float = 0.02,
                   max_steps: int = 600, dt: float = 1.0) -> List[vec]:
    """
    Returns a single continuous path from `start` to `end`
    using two PID-driven points that converge toward each other.
    """
    P, Q  = start, end          # positions
    Vp, Vq = v_start, v_end     # velocities
    Ip, Iq = (0., 0.), (0., 0.) # PID integrals

    path_fwd  : List[vec] = [P]
    path_rev  : List[vec] = [Q]

    for _ in range(max_steps):
        # advance both ends with PID control
        P_next, Vp, Ip = next_point_pid(P, Q, Vp, Ip,
                                        max_accel, Kp, Ki, Kd, dt)
        Q_next, Vq, Iq = next_point_pid(Q, P, Vq, Iq,
                                        max_accel, Kp, Ki, Kd, dt)

        # meet/cross check
        if hypot(P_next[0] - Q_next[0], P_next[1] - Q_next[1]) < epsilon:
            mid = ((P_next[0] + Q_next[0]) / 2,
                   (P_next[1] + Q_next[1]) / 2)
            path_fwd.append(mid)
            path_rev.append(mid)
            break

        path_fwd.append(P_next)
        path_rev.append(Q_next)
        P, Q = P_next, Q_next

    # Merge into one continuous list
    path_rev.reverse()
    path_rev = path_rev[1:]        # drop duplicate midpoint
    return path_fwd + path_rev


# ------------------- demo --------------------------------------------
full = meet_paths_pid((10, 0), (2, 2),
                        v_start=(0.5, 0), 
                        v_end=(-1, 0),
                        max_accel=0.05,
                        Kp=8, Kd=100, Ki=0.00001,
                        epsilon=0.01, max_steps=40)

xs, ys = zip(*full)
px.scatter(x=xs, y=ys)



In [ ]:


# def motion_profiler(
#         L1: line, L2: line,
#         S1: speed, S2: speed,
#         preL1: line, preL2: line,
#         max_accel = 0.01):

#     dist_between = np.sqrt((L2[1][0] - L1[0][0])**2 + (L2[1][1] - L1[0][1])**2)
#     next_point = ()



L1 = ((0, 0), (10, 0))
L2 = ((0, 1), (10, 1))
S1 = (0.1, 0)
S2 = (0.1, 0)

# profile = motion_profiler(L1, L2, S1, S2, L1, L2, 0.01)


In [253]:
import numpy as np
import plotly.express as px

def move_parabola(x1, y1, x2, y2, steps):
    points = []
    for i in range(steps + 1):
        t = i / steps
        ease = 3 * t**2 - 2 * t**3  # smootherstep easing
        x = x1 + (x2 - x1) * ease
        y = y1 + (y2 - y1) * ease
        points.append((x, y))
    return points

points = move_parabola(10, 0, 0, 1, 100)
x_vals, y_vals = zip(*points)
fig = px.scatter(x=x_vals, y=y_vals, title="Parabolic Move Trajectory (Smooth)")
fig.update_traces(mode='lines+markers')
fig.show()
